In [ ]:
# setup Yolo environment
%cd "/content/drive/MyDrive/Colab Notebooks/Bruce/Yolo8-lib"

import os
import sys

os.environ["PYTHONPATH"]="/content/drive/MyDrive/Colab Notebooks/Bruce/Yolo8-lib"
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Bruce/Yolo8-lib/ultralytics')

In [4]:
# change COCO format to YOLO format
from numpy.lib.type_check import imag
import re
import os
import cv2

img_path = "/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/original dataset/train/haze_images"
label_path = "/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/original dataset/train/haze_images_labels"
dest_path = "/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/labels/train"

for filename_jpg in os.listdir(img_path):
  yolo = ""
  img = cv2.imread(os.path.join(img_path, filename_jpg))
  img_H = img.shape[0]
  img_W = img.shape[1]
  filename_txt = os.path.join(label_path, filename_jpg.replace("jpg", "txt"))
  if filename_txt.endswith(".txt"):
    with open(os.path.join(label_path, filename_txt), 'r') as f:
      lines = f.readlines()
      for line in lines:
        x_min, y_min, x_max, y_max = [int(bbox) for bbox in line.split()[1:]]
        x_center = round((x_max + x_min) / 2  / img_W, 4)
        y_center = round((y_max + y_min) / 2 / img_H, 4)
        width = round((x_max - x_min) / img_W, 4)
        height = round((y_max - y_min) / img_H, 4)
        yolo += (f"0 {x_center} {y_center} {width} {height}\n")
    print(yolo)
    print(os.path.join(dest_path, filename_jpg.replace("jpg", "txt")))
    with open(os.path.join(dest_path, filename_jpg.replace("jpg", "txt")), 'w') as f:
      f.write(yolo)

0 0.0051 0.0713 0.0103 0.0467
0 0.3927 0.354 0.1458 0.088
0 0.5713 0.3713 0.0542 0.1347

/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/labels/train/011.txt
0 0.4547 0.2033 0.0423 0.096
0 0.136 0.449 0.1127 0.0793
0 0.1539 0.3293 0.0683 0.0453
0 0.4583 0.4237 0.0298 0.0727
0 0.3607 0.4237 0.0743 0.042
0 0.1396 0.1737 0.1187 0.1167

/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/labels/train/006.txt
0 0.3829 0.1207 0.0255 0.06
0 0.3043 0.1007 0.0602 0.0373
0 0.1377 0.0723 0.0954 0.0473

/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/labels/train/018.txt
0 0.2908 0.0203 0.0439 0.0407
0 0.4799 0.0483 0.032 0.07
0 0.477 0.206 0.0217 0.0533
0 0.4127 0.1937 0.0493 0.03
0 0.2729 0.186 0.0634 0.04
0 0.2846 0.104 0.0401 0.0387

/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/labels/train/009.txt
0 0.1385 0.0833 0.1035 0.0853
0 0.1057 0.275 0.1821 0.0967
0 0.5699 0.3243 0.0417 0.1073
0 0.439 0.2977 0.1

In [2]:
# Run Yolo
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # load a pretrained model
results = model.train(data="/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/Hazy.yaml", epochs=150, pretrained=True, iou=0.7, visualize=True, patience=5)  # train the model
results = model.val()  # evaluate model performance on the validation set

Ultralytics YOLOv8.0.17 🚀 Python-3.9.16 torch-2.0.0+cu117 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/Hazy.yaml, epochs=150, patience=5, batch=16, imgsz=640, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=True, augment=False, agnostic_nms=False, retina_masks=False, classes=None, format=torchscript, keras=False, optimize=False, int8=

In [3]:
# Generate inference results from testing dataset
!chmod 755 bin/yolo
PATH="runs/detect/train31/"
!bin/yolo task=detect mode=predict model='{PATH}/weights/best.pt' save_txt=True conf=0.7 source='/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/images/test'

2023-04-25 03:47:44.943617: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Ultralytics YOLOv8.0.17 🚀 Python-3.9.16 torch-2.0.0+cu117 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/20 /content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/images/test/182.jpg: 544x640 8 vehicles, 56.4ms
image 2/20 /content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/images/test/183.jpg: 544x640 2 vehicles, 7.5ms
image 3/20 /content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/images/test/184.jpg: 544x640 1 vehicle, 7.5ms
image 4/20 /content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/images/test/185.jpg: 544x640 2 vehicles, 7.5ms
image 5/20 /content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/images/test/187.jpg: 544x640 2 vehicles, 7.5ms
image 6/20 /content/drive/MyDrive/Colab Notebooks/CVPR23 Object in

In [ ]:
# run local tensorboard
!ls "/content/drive/MyDrive/Colab Notebooks/Bruce/Yolo8-lib/runs/detect/train8"
%cd "/content/drive/MyDrive/Colab Notebooks/Bruce/Yolo8-lib/"
%tensorboard --logdir "runs/detect/train24"

In [ ]:
# run cloud tensorboard
!tensorboard dev upload --logdir runs/detect/train27 --name "150 epochs exr scratch unfreeze normalize=true pretrain=false fishnofish #normalize=true"

2023-03-28 06:54:57.740455: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /content/drive/MyDrive/Colab Notebooks/Bruce/Yolo8-lib/cv2/../../lib64:/usr/lib64-nvidia
2023-03-28 06:54:57.740653: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /content/drive/MyDrive/Colab Notebooks/Bruce/Yolo8-lib/cv2/../../lib64:/usr/lib64-nvidia
2023-03-28 06:54:57.740684: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
Upload started and will c